In [ ]:
%spark.pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, expr, countDistinct
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType
verdadero='true'

input_bucket = 's3://data-emr-motor/movies_parquet/'

# to read parquet file
df_keywords= spark.read.option("header",verdadero).parquet(input_bucket + 'movies_metadata/')
df_keywords.createOrReplaceGlobalTempView("processed_movies_metadata")
df_keywords2= spark.read.option("header",verdadero).parquet(input_bucket + 'ratings_small/')
df_keywords2.createOrReplaceGlobalTempView("processed_ratings_small")
df_keywords3= spark.read.option("header",verdadero).parquet(input_bucket + 'ratings/')
df_keywords3.createOrReplaceGlobalTempView("processed_ratings")
df_keywords4= spark.read.option("header",verdadero).parquet(input_bucket +  'keywords/')
df_keywords4.createOrReplaceGlobalTempView("processed_keywords")
df_keywords5= spark.read.option("header",verdadero).parquet(input_bucket + 'credits/')
df_keywords5.createOrReplaceGlobalTempView("processed_credits")


In [ ]:
%spark.pyspark
spark.catalog.listTables('default')

In [ ]:
%spark.pyspark

def recomendacion(palbra_clave1, palabra_clave2, actor, actor2, actor3, cantidad_recomendaciones=3):
    sql = """
        WITH movie_picks AS (
           SELECT m.title,
                date(m.release_date) AS release_date,
                m.popularity, m.title,
                count(r.movieid) AS ratings_count,
                round(avg(r.rating), 3) AS avg_rating
           FROM processed_movies_metadata AS m
           LEFT JOIN processed_ratings AS r ON m.id = r.movieid
           LEFT JOIN processed_keywords AS k ON m.id = k.id
           LEFT JOIN processed_credits AS c ON m.id = c.id
           WHERE (
                    lower(k.keywords) LIKE '%{0}%'
                    OR lower(k.keywords) LIKE '%{1}%'
                )
                AND (
                    lower(c.cast) LIKE '%{2}%'
                    OR lower(c.cast) LIKE '%{3}%'
                    OR lower(c.cast) LIKE '%{4}%'
                )
            GROUP BY m.title,
                m.release_date,
                m.popularity
            )
            SELECT title,
            ntile({5}) OVER (
                ORDER BY avg_rating DESC
            ) AS rank,
            avg_rating,
            ratings_count,
            popularity,
            release_date
        FROM movie_picks
        WHERE avg_rating > 0
        ORDER BY rank,
            avg_rating DESC,
            ratings_count DESC
       """.format(palbra_clave1, palabra_clave2, actor, actor2, actor3, cantidad_recomendaciones)
       
    return spark.sql(sql)


In [ ]:
%spark.pyspark

palbra_clave1 = 'artificial intelligence'
palabra_clave2 = 'robot'
actor = 'arnold schwarzenegger'
actor2 = 'jacki'
actor3 =  'will smith'

df_ratings = recomendaciones(palbra_clave1, palabra_clave2, actor, actor2, actor3)



In [ ]:
%spark.pyspark
df_ratings.show()


In [ ]:
%spark.pyspark
sql = f"""
    SELECT ntile(10) OVER (ORDER BY r.rating DESC) AS rank,
           avg(r.rating) AS avg_rating,
           count(r.movieid) AS ratings_count,
           round(avg(cast(m.popularity AS DOUBLE)), 2) AS popularity
       FROM processed_movies_metadata AS m
           LEFT JOIN processed_ratings_small AS r ON m.id = r.movieid
       WHERE r.rating IS NOT NULL
           AND from_unixtime(r.timestamp) >= to_timestamp('2000-01-01')
       GROUP BY r.rating
   """

df_ratings = spark.sql(sql)

In [ ]:
%spark.pyspark
df_ratings.show()
